BOUNDING BOX DISPARITY: 3D METRICS FOR OBJECT DETECTION WITH FULL DEGREE OF FREEDOM

Artigo: https://arxiv.org/pdf/2207.03720

Código: https://github.com/M-G-A/3D-Metrics/tree/main

Este notebook demonstra como avaliar a precisão de detecções 3D utilizando a função evaluate, que calcula métricas como Intersection over Union (IoU), Volume-to-volume distance (V2V), e Bounding Box Disparity (BBD).

## Métricas

### Volumetric IoU
A métrica **Intersection over Union (IoU)** volumétrica é uma medida utilizada na avaliação de sobreposição entre duas bounding boxes tridimensionais. O IoU é definido como a razão entre o volume da interseção das duas bbs (cubóides) e o volume da união delas.

#### Etapas para calcular o IoU volumétrico:

1. **Definição dos Pontos de Interesse (POI)**:
   - Os pontos de interesse são definidos como:
     - Canto da primeira cuboide dentro da segunda.
     - Canto da segunda cuboide dentro da primeira.
     - Interseções das arestas da primeira cuboide com os planos da segunda.
     - Interseções das arestas da segunda cuboide com os planos da primeira.
       
2. **Verificação da Validade dos Pontos**:
   - Cada ponto de interesse deve ser transformado no sistema de coordenadas de uma das cuboides (T1) e verificado para determinar se está dentro dos limites da cuboide. Os pontos válidos são aqueles cujas coordenadas estão entre -0.5 e 0.5, após a transformação.

3. **Cálculo do Volume**:
   - Para os pontos válidos, um envoltório convexo (convex hull) é construído. Se não for possível construir esse envoltório (por exemplo, se não houver pontos válidos ou se todos os pontos estiverem em um plano), então considera-se que não há interseção e o IoU é zero.
   - Caso contrário, o volume da interseção é calculado a partir do envoltório convexo. O volume da união é obtido pela soma dos volumes das duas cuboides, subtraindo o volume da interseção.

Um IoU próximo de 1 indica uma boa sobreposição entre as bbs, enquanto valores próximos de 0 sugerem pouca ou nenhuma sobreposição.

### Volume-to-volume distance (V2V)

A métrica de Volume-to-Volume Distance (V2V) calcula a menor distância entre duas bounding boxes tridimensionais, representadas como cuboides. Essa distância é definida como a menor distância ( ds ) entre pontos de interesse (PPOIs) que estão nos contornos (hulls) das duas caixas.

#### Definição de Pontos-Pares de Interesse (PPOIs)

Os pontos de interesse são definidos a partir das seguintes considerações:

1. **Projeções de Canto**: As projeções retangulares dos cantos de um cubo sobre as faces do outro cubo e vice-versa.
2. **Projeções em Arestas**: Os cantos de um cubo em relação às arestas do outro cubo.
3. **Distância entre Arestas**: Distâncias entre pares de arestas de ambos os cubos.
4. **Distância entre Cantos**: Distâncias entre pares de cantos de ambos os cubos.

Esses pares são utilizados para calcular a menor distância possível, considerando que a superfície dos cubos é um conjunto infinito de pontos, mas as avaliações são feitas em um conjunto discreto.

#### Projeções e Cálculos

Para calcular a menor distância entre pontos e superfícies ou arestas, o método utiliza projeções geométricas:

- **Projeção Ponto-Plano**: A distância entre um ponto e um plano é obtida pela projeção do ponto no plano. A validade da projeção é verificada se ela cai dentro dos limites do plano.
  
- **Projeção Ponto-Linha**: A distância entre um ponto e uma linha é obtida a partir da projeção do ponto na linha. Novamente, é verificado se a projeção está dentro do segmento da linha.

- **Projeção Linha-Linha**: A distância entre duas linhas é calculada considerando a distância mínima entre elas.

#### Cálculo da Distância

No cálculo final, o algoritmo determina a menor distância entre todos os pares de pontos de interesse encontrados.

### Bounding Box Disparity (BBD)

A **Bounding Box Disparity (BBD)** é uma métrica que combina o **Intersection over Union (IoU)** e a **Volume-to-Volume Distance (V2V)** para avaliar a similaridade entre duas bounding boxes. Essa métrica é particularmente útil em cenários onde as caixas podem não se sobrepor, mas ainda assim é necessário medir a proximidade ou a diferença entre elas.

#### Justificativa

O IoU é uma métrica amplamente utilizada que mede a sobreposição entre duas bbs, resultando em valores entre 0 e 1, onde:
- **1** indica uma correspondência total (ou seja, as bbs se sobrepõem completamente).
- **0** indica que não há sobreposição.

Entretanto, o IoU não pode distinguir entre bbs que estão próximas uma da outra sem se sobrepor. Para resolver essa limitação, a métrica BBD combina IoU e V2V da seguinte maneira:

BBD = 1 − IoU + v2v

#### Comportamento da Métrica

- **Quando as caixas se sobrepõem**: O valor de IoU aumenta, fazendo com que a BBD diminua. Se houver uma correspondência total, a BBD será minimizada, refletindo alta similaridade.
- **Quando não há sobreposição**: A distância entre as bbs (v2v) aumenta, resultando em um aumento na BBD. Assim, a BBD se torna uma métrica contínua que quantifica a (des)similaridade entre as bbs, onde valores mais altos indicam maior disparidade.

## Função evaluate

A função `evaluate` recebe dois parâmetros principais: uma instância de **bounding box ground truth** e uma **predição**.

1. **Ground Truth Bounding Box** (`gt_instance`): 
   Dicionário contendo as informações completas de uma bounding box no formato do ground truth. Exemplo:
   ```python
   {
       'trackId': '21658603-47f6-4ba4-9864-0f789229e6f8',
       'trackName': '2',
       'groups': [],
       'contour': {
           'center3D': {'x': 50.4931, 'y': 4.6632, 'z': 0.2638},
           'pointN': 7,
           'points': [],
           'rotation3D': {'x': 0, 'y': 0, 'z': 3.1106},
           'sensorDistance': 50.71,
           'size3D': {'x': 9.765, 'y': 3.4457, 'z': 3.4716}
       },
       'modelConfidence': None,
       'modelClass': '',
       'classVersion': 1,
       'isValid': None,
       'note': None,
       'start': None,
       'end': None,
       'deviceName': None,
       'deviceFrame': None,
       'bevFrameName': None,
       'index': None,
       'role': None,
       'content': None,
       'id': '5e320a9c-767b-4e2e-bbbd-5ab0bc5281a4',
       'type': '3D_BOX',
       'classId': 2184728,
       'className': 'bus',
       'classNumber': None,
       'classValues': [],
       'createdAt': 1724768778000,
       'createdBy': 1680339
   }
   
1. **Prediction** (`prediction_instance`): 
Dicionário que representa a bounding box predita pelo modelo. Exemplo:

    ```python
    {
        'center3D': {'x': -11.5158, 'y': 43.9325, 'z': -2.6742},
        'rotation3D': {'x': 5.0805e-09, 'y': -1.6591e-09, 'z': 2.0792},
        'size3D': {'x': 4.6225, 'y': 1.9612, 'z': 1.7235},
        'confidence': 0.5224,
        'labelId': 0,
        'type': '3D_BOX'
    }


A função irá retornar as métricas Volumetric IoU, Volume-to-volume distance (V2V), e Bounding Box Disparity (BBD) para as duas bounding boxes.

## Code example

In [17]:
import os, json
from evaluate import evaluate

In [18]:
def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [22]:
# Obtem todos os arquivos ground truths
folder_path_gt = 'GT/GT'
folder_path_results = 'results/results'

gt_names = os.listdir(folder_path_gt)

gt_jsons = []
pred_jsons = []

# Obtem todos os jsons ground truths e predictions
for gt_name in gt_names:
    gt_path = folder_path_gt + '/' + gt_name
    pred_path = folder_path_results + '/' + gt_name

    if not "json" in gt_path:
        continue
        
    gt_json = load_json_file(gt_path)
    pred_json = load_json_file(pred_path)

    gt_jsons.append(gt_json)
    pred_jsons.append(pred_json)

Função que recebe o json de todos os ground truths de um arquivo e o json de todas as predições do mesmo arquivo, e calcula as métricas para cada combinação deles.

In [20]:
def processar_metricas(ground_truths, predicoes):
    resultados = []

    # Itera sobre cada ground truth do arquivo
    for instancia_gt in gt['instances']:
        gt_track_id = instancia_gt['trackId']

        # Para cada ground truth, faz a evaluation para cada predição
        for key, predicao_data in predicoes.items():
            for predicao in predicao_data['detections']:
                pred_confidence = predicao['confidence']

                metricas = evaluate(instancia_gt, predicao)

                # Organizar os resultados
                resultado = {
                    'ground_truth': {
                        'trackId': gt_track_id,
                    },
                    'predicao': {
                        'confidence': pred_confidence,
                    },
                    'metricas': metricas
                }

                # Adicionar resultado à lista
                resultados.append(resultado)

    return resultados

In [ ]:
resultados_finais = []

for gt, pred in zip(gt_jsons, pred_jsons):
    gt = gt[0]
    resultados_finais.append({
        'dataId': gt['dataId'],
        'metricas': processar_metricas(gt, pred['signalsReceived'])
    })

with open("resultados_metricas.json", "w") as f:
    json.dump(resultados_finais, f, indent=4)